# Начало

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


# Импорт библиотек

In [ ]:
!pip install simpletransformers transformers
!pip install jsonlines

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import json
import pickle
import difflib
import pandas as pd
import numpy as np
import torch
from collections import Counter
from sklearn.metrics import *
import jsonlines

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

# Загрузка данных

In [ ]:
train = []
with open('/content/drive/MyDrive/rucos_train.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text.lower()
        entities = []
        for l in source['passage']['entities']:
            entities.append(text[l['start']:l['end']].lower())
        for q in source["qas"]:
                query = q["query"]
                idx = q["idx"]
                query = query.lower()
                answers = [x["text"] for x in q["answers"]]
                start = [x['start'] for x in q['answers']]
                end = [x['end'] for x in q['answers']]
                answersa = []
                for s in range(len(answers)):
                    answersa = [{'text': answers[s].lower(), 'answer_start':start[s]}]
                    qasa = [{'is_impossible':False,'question':query,'answers':answersa,'id':idx }]
                    train.append({'context':text, 'qas': qasa})

train_notlower = []
with open('/content/drive/MyDrive/rucos_train.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text
        entities = []
        for l in source['passage']['entities']:
            entities.append(text[l['start']:l['end']].lower())
        for q in source["qas"]:
                query = q["query"]
                idx = q["idx"]
                query = query
                answers = [x["text"] for x in q["answers"]]
                start = [x['start'] for x in q['answers']]
                end = [x['end'] for x in q['answers']]
                answersa = []
                for s in range(len(answers)):
                    answersa = [{'text': answers[s], 'answer_start':start[s]}]
                    qasa = [{'is_impossible':False,'question':query,'answers':answersa,'id':idx }]
                    train_notlower.append({'context':text, 'qas': qasa})

test = []
with open('/content/drive/MyDrive/rucos_test.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text.lower()
        for q in source["qas"]:
                query = q["query"]
                idx = q["idx"]
                query = query.lower()
                answersa = []
                qasa = [{'is_impossible':False,'question':query,'id':idx }]
                test.append({'context':text, 'qas': qasa})

test_notlower = []
with open('/content/drive/MyDrive/rucos_test.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text
        for q in source["qas"]:
                query = q["query"]
                idx = q["idx"]
                query = query
                answersa = []
                qasa = [{'is_impossible':False,'question':query,'id':idx }]
                test.append({'context':text, 'qas': qasa})

dev_notlower = []
with open('/content/drive/MyDrive/rucos_val.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text
        for q in source["qas"]:
                query = q["query"]
                idx = q["idx"]
                query = query
                answers = [x["text"] for x in q["answers"]]
                start = [x['start'] for x in q['answers']]
                end = [x['end'] for x in q['answers']]
                answersa = []
                answersa = [{'text': answers[0], 'answer_start':start[0]}]
                qasa = [{'is_impossible':False,'question':query,'answers':answersa,'id':idx }]
                dev_notlower.append({'context':text, 'qas': qasa})

dev = []
with open('/content/drive/MyDrive/rucos_val.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text.lower()
        for q in source["qas"]:
                query = q["query"]
                idx = q["idx"]
                query = query.lower()
                answers = [x["text"] for x in q["answers"]]
                start = [x['start'] for x in q['answers']]
                end = [x['end'] for x in q['answers']]
                answersa = []
                answersa = [{'text': answers[0], 'answer_start':start[0]}]
                qasa = [{'is_impossible':False,'question':query,'answers':answersa,'id':idx }]
                dev.append({'context':text, 'qas': qasa})

In [ ]:
def compute_f1(true, pred):
    tr = tokenizer.tokenize(str(true).lower())
    pr = tokenizer.tokenize(str(pred).lower())
    # f1 = int(tr == pr)
    common = Counter(tr) & Counter(pr)
    num_same = sum(common.values())
    if len(tr) == 0 or len(pr) ==0:
        return int(tr == pr)
    if num_same == 0:
        return 0
    presicion = 1 * num_same / len(pr)
    recall = 1 * num_same / len(tr)
    f1 = (2 * presicion * recall) / (presicion + recall) 
    return f1

# Обучение моделей (у нас тут пачка бертов из simple transformers, поэтому оно очень разнообразное :) но если хотите - можете запустить; а так можете посмотреть и перейти к следующему пункту, где мы загружаем все эти модели (уже обученные))

Модель №1 (он же - model_rubert_low ): 

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 10
model_args.num_train_epochs=5
model_args.max_seq_length = 512
model_args.evaluate_during_training = False
model_args.use_cuda = True
model_args.no_cache = True
model_args.save_steps = 25000

In [ ]:
model = QuestionAnsweringModel(
    "bert", "DeepPavlov/rubert-base-cased-sentence", args=model_args
)

In [ ]:
model.train_model(train)

In [ ]:
filename = '/content/drive/MyDrive/New_models/model_rubert_low.pkl'
pickle.dump(model, open(filename, 'wb'))

Модель №2 (он же - bert_dp_lower):

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 10
model_args.num_train_epochs=3
model_args.max_seq_length = 512
model_args.evaluate_during_training = False
model_args.use_cuda = True
model_args.use_multiprocessing = False
model_args.overwrite_output_dir = True

In [ ]:
model = QuestionAnsweringModel(
    "bert", "DeepPavlov/bert-base-multilingual-cased-sentence", args=model_args
)


In [ ]:
model.train_model(train)


In [ ]:
import pickle
with open('dpmlbert_cased_lower.pkl', 'wb') as f:
    pickle.dump(model, f)

Модель №3 (он же - distilbert_lower ):

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 10
model_args.num_train_epochs=3
model_args.max_seq_length = 512
model_args.evaluate_during_training = False
model_args.use_cuda = True
model_args.overwrite_output_dir = True

In [ ]:
model = QuestionAnsweringModel(
    "distilbert", "distilbert-base-multilingual-cased", args=model_args
)

In [ ]:
model.train_model(train)

In [ ]:
import pickle
with open('distilbert_cased_lower.pkl', 'wb') as f:
    pickle.dump(model, f)

Модель №4 (он же - distilbert_notlower):

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 10
model_args.num_train_epochs=3
model_args.max_seq_length = 512
model_args.evaluate_during_training = False
model_args.use_cuda = True
model_args.overwrite_output_dir = True

In [ ]:
model = QuestionAnsweringModel(
    "distilbert", "distilbert-base-multilingual-cased", args=model_args
)

In [ ]:
model.train_model(train_notlower)

In [ ]:
import pickle
with open('distilbert_cased_notlower.pkl', 'wb') as f:
    pickle.dump(model, f)

Модель №5 (он же - dprubertconv_cased_lower):

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 10
model_args.num_train_epochs=3
model_args.max_seq_length = 512
model_args.evaluate_during_training = False
model_args.use_cuda = True
model_args.overwrite_output_dir = True

In [ ]:
model = QuestionAnsweringModel(
    "bert", "DeepPavlov/rubert-base-cased-conversational", args=model_args
)

In [ ]:
model.train_model(train)

In [ ]:
import pickle
with open('dprubertconv_cased_lower.pkl', 'wb') as f:
    pickle.dump(model, f)

Модель №6 (он же - dprubertconv_cased_notlower):

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 10
model_args.num_train_epochs=3
model_args.max_seq_length = 512
model_args.evaluate_during_training = False
model_args.use_cuda = True
model_args.overwrite_output_dir = True

In [ ]:
model = QuestionAnsweringModel(
    "bert", "DeepPavlov/rubert-base-cased-conversational", args=model_args
)

In [ ]:
model.train_model(train_notlower)

In [ ]:
import pickle
with open('dprubertconv_cased_notlower.pkl', 'wb') as f:
    pickle.dump(model, f)

Модель №7 (он же - finalized_model_bert):

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 10
model_args.num_train_epochs=3
model_args.max_seq_length = 512
model_args.evaluate_during_training = False
model_args.use_cuda = True

In [ ]:
model = QuestionAnsweringModel(
    "bert", "DeepPavlov/rubert-base-cased-sentence", args=model_args
)

In [ ]:
model.train_model(train)

In [ ]:
filename = '/content/drive/MyDrive/New_models/finalized_model_bert.pkl'
pickle.dump(model, open(filename, 'wb'))

Модель №8 (он же bert_f_xquad):
её тренил мой сокомандник, и так как её скор сильно хуже других (0.6 всего лишь) - мы решили её не использовать в ансамбле и мой сокомандник не кинул мне её код (просто чтобы потом не было вопрос а что это за закоменченная штуковина)

На этом вроде бы все 
(но если вдруг я что-то пропустил - пишите, я мог что-то пропустить)

# Загрузка обученных нами моделей (для ансамблирования)

In [ ]:
# with open('/content/drive/MyDrive/New_models/bert_f_xquad.pkl', 'rb') as f:
#     bert_xquad_notlower = pickle.load(f)
# with open('/content/drive/MyDrive/New_models/distilbert_cased_notlower.pkl', 'rb') as f:
#     distilbert_notlower = pickle.load(f)
with open('/content/drive/MyDrive/dpmlbert_cased_lower.pkl', 'rb') as f:
    bert_dp_lower = pickle.load(f)
with open('/content/drive/MyDrive/dprubertconv_cased_lower.pkl', 'rb') as f:
    rubert_conv_dp_lower = pickle.load(f)
with open('/content/drive/MyDrive/dprubertconv_cased_notlower.pkl', 'rb') as f:
    rubert_conv_dp_notlower = pickle.load(f)
with open('/content/drive/MyDrive/model_rubert_low.pkl', 'rb') as f:
    rubert_lower = pickle.load(f)
with open('/content/drive/MyDrive/model_rubert_sentence_low.pkl', 'rb') as f:
    rubert_sen_lower = pickle.load(f)


with open('/content/drive/MyDrive/model_rubert_no.pkl', 'rb') as f:
    rubert_sen_dp_lower = pickle.load(f)

with open('/content/drive/MyDrive/finalized_model_bert.pkl', 'rb') as f:
    bert_fin = pickle.load(f)


with open('/content/drive/MyDrive/model_bert_multi_low.pkl', 'rb') as f:
    bert_multi = pickle.load(f)

    # /content/drive/MyDrive/New_models/finalized_model_bert.pkl
# /content/drive/MyDrive/New_models/dpmlbert_cased_notlower.pkl

# А дальше начинается сущий ад и куча методов ансамблирования предиктов бертов, которые мы придумали

In [ ]:
# получение предиктов каждого берта


# preds_bert_xquad_notlower = bert_xquad_notlower.predict(test)
# preds_distilbert_notlower = distilbert_notlower.predict(test)
preds_bert_dp_lower = bert_dp_lower.predict(dev)
preds_rubert_conv_dp_lower = rubert_conv_dp_lower.predict(dev)
preds_rubert_conv_dp_notlower = rubert_conv_dp_notlower.predict(dev)
preds_rubert_lower = rubert_lower.predict(dev)
preds_rubert_sen_lower = rubert_sen_lower.predict(dev)
preds_bert_fin = bert_fin.predict(dev)
preds_bert_multi = bert_multi.predict(dev)
preds_bert_fin = bert_fin.predict(dev)
preds_rubert_sen_dp_lower = rubert_sen_dp_lower.predict(dev)

add example index and unique id: 100%|██████████| 7577/7577 [00:00<00:00, 737651.54it/s]


Running Prediction:   0%|          | 0/949 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7577/7577 [00:00<00:00, 766509.28it/s]


Running Prediction:   0%|          | 0/948 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7577/7577 [00:00<00:00, 782013.37it/s]


...

Running Prediction:   0%|          | 0/949 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7577/7577 [00:00<00:00, 765438.51it/s]


Running Prediction:   0%|          | 0/948 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7577/7577 [00:00<00:00, 754731.68it/s]


Running Prediction:   0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
# получение вариантов ответа и самих ответов для проверки точности модели/ансамбля на валидации

entities_m = []
with open('/content/drive/MyDrive/rucos_val.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text.lower()
        entities = []
        for l in source['passage']['entities']:
            entities.append(text[l['start']:l['end']].lower())
        entities_m.append(entities)                   

real_answ_m = []
with open('/content/drive/MyDrive/rucos_val.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text.lower()
        for q in source["qas"]:
            answers = [x["text"] for x in q["answers"]]
        real_answ_m.append(answers[0].lower())

In [ ]:
# форматирование предиктов чтобы потом впихнуть их в довольно удобный pd.DataFrame
answers_bert_dp_lower = []
for i in range(len(preds_bert_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_dp_lower.append([preds_bert_dp_lower[0][i]['answer'][0], preds_bert_dp_lower[1][i]['probability'][0]])

answers_rubert_conv_dp_lower = []
for i in range(len(preds_rubert_conv_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_conv_dp_lower.append([preds_rubert_conv_dp_lower[0][i]['answer'][0], preds_rubert_conv_dp_lower[1][i]['probability'][0]])

answers_rubert_conv_dp_notlower = []
for i in range(len(preds_rubert_conv_dp_notlower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_conv_dp_notlower.append([preds_rubert_conv_dp_notlower[0][i]['answer'][0], preds_rubert_conv_dp_notlower[1][i]['probability'][0]])

answers_rubert_lower = []
for i in range(len(preds_rubert_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_lower.append([preds_rubert_lower[0][i]['answer'][0], preds_rubert_lower[1][i]['probability'][0]])

answers_rubert_sen_lower = []
for i in range(len(preds_bert_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_sen_lower.append([preds_rubert_sen_lower[0][i]['answer'][0], preds_rubert_sen_lower[1][i]['probability'][0]])

answers_bert_fin = []
for i in range(len(preds_bert_fin[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_fin.append([preds_bert_fin[0][i]['answer'][0], preds_bert_fin[1][i]['probability'][0]])

answers_bert_multi = []
for i in range(len(preds_bert_multi[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_multi.append([preds_bert_multi[0][i]['answer'][0], preds_bert_multi[1][i]['probability'][0]])

answers_bert_fin = []
for i in range(len(preds_bert_fin[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_fin.append([preds_bert_fin[0][i]['answer'][0], preds_bert_fin[1][i]['probability'][0]])

answers_rubert_sen_dp_lower = []
for i in range(len(preds_rubert_sen_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_sen_dp_lower.append([preds_rubert_sen_dp_lower[0][i]['answer'][0], preds_rubert_sen_dp_lower[1][i]['probability'][0]])

In [ ]:
# обработка предиктов бертов (что они непосредственно предсказали)
def get_answers(ans):
    rets = []
    for i,k in enumerate(entities_m[:]):
        matches = difflib.get_close_matches(ans[i], k, cutoff=0.4)
        if matches:
            rets.append(matches[0])
        else:
            rets.append(ans[i])
    return rets



real_answers_bert_dp_lower = get_answers(answers_bert_dp_lower)
real_answers_rubert_conv_dp_lower = get_answers(answers_rubert_conv_dp_lower)
real_answers_rubert_conv_dp_notlower =  get_answers(answers_rubert_conv_dp_notlower)
real_answers_rubert_lower = get_answers(answers_rubert_lower)
real_answers_rubert_sen_lower = get_answers(answers_rubert_sen_lower)
real_answers_bert_fin = get_answers(answers_bert_fin)
real_answers_bert_multi = get_answers(answers_bert_multi)
real_answers_bert_fin = get_answers(answers_bert_fin)
real_answers_rubert_sen_dp_lower = get_answers(answers_rubert_sen_dp_lower)


In [ ]:
# создание удобного датафрейма
answers__ = pd.DataFrame({
                          'bert_dp_lower':real_answers_bert_dp_lower, 'rubert_conv_dp_lower':real_answers_rubert_conv_dp_lower,
                           'rubert_conv_dp_notlower':real_answers_rubert_conv_dp_notlower,'rubert_lower':real_answers_rubert_lower,
                           'rubert_sen_lower':real_answers_rubert_sen_lower,'bert_fin':real_answers_bert_fin,
                          'bert_multi':real_answers_bert_multi,'bert_fin':real_answers_bert_fin, 'rubert_sen_dp_lower':real_answers_rubert_sen_dp_lower})

In [ ]:
#сохранение его в отдельный файл, чтобы не запускать 10 раз в день те 5 ячеек, которые выше
answers__.to_csv('Answers_from_every_model.csv')

In [ ]:
answers__ = pd.read_csv('Answers_from_every_model.csv')

In [ ]:
answers__

,Unnamed: 0,bert_xquad_notlower,distilbert_notlower,bert_dp_lower,rubert_conv_dp_lower,rubert_conv_dp_notlower,rubert_sen_dp_lower
0,0,германии,германии,германии,германии,германии,германии
1,1,хамас,хамас,рф,рф,хамас,рф
2,2,ташкента,ташкента,ташкента,ташкента,ташкента,ташкента
3,3,heckler & koch,heckler & koch,heckler & koch,heckler & koch,сми,heckler & koch
4,4,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер
...,...,...,...,...,...,...,...
7572,7572,ирана,ближнем востоке,ирана,иран,ирана,ирана
7573,7573,николаенко,николаенко,николаенко,николаенко,николаенко,николаенко
7574,7574,халифа хафтар,халифа хафтар,халифа хафтар,халифа хафтар,хафтар,халифа хафтар
7575,7575,халифы хафтара,халифы хафтара,халифы хафтара,хафтара,россии,хафтара


In [ ]:
# это один из наших методов ансамблирования 
# P.S. уважаемые программисты, не делайте сразу фейспалмы при виде сие творения
# согласен, это сложно назвать нормальным ансамблем
# но оно повышает точность и использует несколько моделей :)
real_an = []
for i in range(len(answers__)):
    st = answers__.iloc[i]
    s = Counter([x[0].lower() for x in st])
    for x in s:
        if s[x] >= 4:
            real_an.append(x)
            break
    else:
        real_an.append(sorted(answers__.iloc[i], key=lambda x: -x[1])[0][0].lower())

IndexError: ignored

In [ ]:
# валидация 1го ансамбля (подсчет f1 у нас кривой, в системе всегда на 0.03-0.09 выше)
f1s = []
for i in range(len(real_an)):
    f1s.append(compute_f1(real_answ_m[i],list(real_an)[i]))
print(sum(f1s) / len(f1s))

0.7607328566613303


In [ ]:
# формирование 1го сабмита (если не ошибаюсь - этот на 0.8143 примерно)
answq = []

for i,l in enumerate(answers_mul):
    a = {'idx': i, 'text':str(l)}
    answq.append(a)

with jsonlines.open('res.jsonl', mode='w') as writer:
    writer.write_all(a)


In [ ]:
# загрузка сабмита
from google.colab import files
files.download('res.jsonl')

In [ ]:
f1s = []
for i in range(len(real_an)):
    f1s.append(compute_f1(real_answ_m[i],list(real_an)[i]))
print(sum(f1s) / len(answers_mul))

0.7561203402168413


In [ ]:
# еще один импорт моделей, не знаю зачем
# но моему сокоманднику так было удобнее


# with open('/content/drive/MyDrive/New_models/bert_f_xquad.pkl', 'rb') as f:
#     bert_xquad_notlower = pickle.load(f)
# with open('/content/drive/MyDrive/New_models/distilbert_cased_notlower.pkl', 'rb') as f:
#     distilbert_notlower = pickle.load(f)
with open('/content/drive/MyDrive/New_models/dpmlbert_cased_notlower.pkl', 'rb') as f:
    bert_dp_lower = pickle.load(f)
with open('/content/drive/MyDrive/New_models/dprubertconv_cased_lower.pkl', 'rb') as f:
    rubert_conv_dp_lower = pickle.load(f)
with open('/content/drive/MyDrive/New_models/dprubertconv_cased_notlower.pkl', 'rb') as f:
    rubert_conv_dp_notlower = pickle.load(f)
with open('/content/drive/MyDrive/New_models/model_rubert_low.pkl', 'rb') as f:
    rubert_lower = pickle.load(f)
with open('/content/drive/MyDrive/New_models/model_rubert_sentence_low.pkl', 'rb') as f:
    rubert_sen_lower = pickle.load(f)


with open('/content/drive/MyDrive/New_models/model_rubert_no.pkl', 'rb') as f:
    rubert_sen_dp_lower = pickle.load(f)

with open('/content/drive/MyDrive/New_models/finalized_model_bert.pkl', 'rb') as f:
    bert_fin = pickle.load(f)


with open('/content/drive/MyDrive/New_models/model_bert_multi_low.pkl', 'rb') as f:
    bert_multi = pickle.load(f)

    
with open('/content/drive/MyDrive/New_models/finalized_model_bert.pkl', 'rb') as f:
    bert_fin = pickle.load(f)


    # /content/drive/MyDrive/New_models/finalized_model_bert.pkl
# /content/drive/MyDrive/New_models/dpmlbert_cased_notlower.pkl

0.7343612065203743


In [ ]:
# preds_bert_xquad_notlower = bert_xquad_notlower.predict(test)
# preds_distilbert_notlower = distilbert_notlower.predict(test)
preds_bert_dp_lower = bert_dp_lower.predict(test)
preds_rubert_conv_dp_lower = rubert_conv_dp_lower.predict(test)
preds_rubert_conv_dp_notlower = rubert_conv_dp_notlower.predict(test)
preds_rubert_lower = rubert_lower.predict(test)
preds_rubert_sen_lower = rubert_sen_lower.predict(test)
preds_bert_fin = bert_fin.predict(test)
preds_bert_multi = bert_multi.predict(test)
preds_bert_fin = bert_fin.predict(test)
preds_rubert_sen_dp_lower = rubert_sen_dp_lower.predict(test)

add example index and unique id: 100%|██████████| 7257/7257 [00:00<00:00, 737677.87it/s]


Running Prediction:   0%|          | 0/910 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7257/7257 [00:00<00:00, 677651.32it/s]


Running Prediction:   0%|          | 0/908 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7257/7257 [00:00<00:00, 801972.50it/s]


...

Running Prediction:   0%|          | 0/910 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7257/7257 [00:00<00:00, 916947.25it/s]


Running Prediction:   0%|          | 0/908 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 7257/7257 [00:00<00:00, 913096.27it/s]


Running Prediction:   0%|          | 0/908 [00:00<?, ?it/s]

In [ ]:
# загрузка вариантов ответа из теста
# следующие несколько ячеек - обработка теста + сабмит
entities_m = []
with open('rucos_test.jsonl', "r") as fhe:
    for i,fh in enumerate(fhe):
        source = json.loads(fh)
        text = source["passage"]["text"]
        text = text.lower()
        entities = []
        for l in source['passage']['entities']:
            entities.append(text[l['start']:l['end']].lower())
        entities_m.append(entities)                   

In [ ]:

answers_bert_dp_lower = []
for i in range(len(preds_bert_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_dp_lower.append([preds_bert_dp_lower[0][i]['answer'][0], preds_bert_dp_lower[1][i]['probability'][0]])

answers_rubert_conv_dp_lower = []
for i in range(len(preds_rubert_conv_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_conv_dp_lower.append([preds_rubert_conv_dp_lower[0][i]['answer'][0], preds_rubert_conv_dp_lower[1][i]['probability'][0]])

answers_rubert_conv_dp_notlower = []
for i in range(len(preds_rubert_conv_dp_notlower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_conv_dp_notlower.append([preds_rubert_conv_dp_notlower[0][i]['answer'][0], preds_rubert_conv_dp_notlower[1][i]['probability'][0]])

answers_rubert_lower = []
for i in range(len(preds_rubert_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_lower.append([preds_rubert_lower[0][i]['answer'][0], preds_rubert_lower[1][i]['probability'][0]])

answers_rubert_sen_lower = []
for i in range(len(preds_bert_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_sen_lower.append([preds_rubert_sen_lower[0][i]['answer'][0], preds_rubert_sen_lower[1][i]['probability'][0]])

answers_bert_fin = []
for i in range(len(preds_bert_fin[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_fin.append([preds_bert_fin[0][i]['answer'][0], preds_bert_fin[1][i]['probability'][0]])

answers_bert_multi = []
for i in range(len(preds_bert_multi[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_multi.append([preds_bert_multi[0][i]['answer'][0], preds_bert_multi[1][i]['probability'][0]])

answers_bert_fin = []
for i in range(len(preds_bert_fin[0])):
    # print(answ[0][i]['answer'][0])
    answers_bert_fin.append([preds_bert_fin[0][i]['answer'][0], preds_bert_fin[1][i]['probability'][0]])

answers_rubert_sen_dp_lower = []
for i in range(len(preds_rubert_sen_dp_lower[0])):
    # print(answ[0][i]['answer'][0])
    answers_rubert_sen_dp_lower.append([preds_rubert_sen_dp_lower[0][i]['answer'][0], preds_rubert_sen_dp_lower[1][i]['probability'][0]])

In [ ]:
def get_answers(ans):
    rets = []
    for i,k in enumerate(entities_m[:]):
        matches = difflib.get_close_matches(ans[i], k, cutoff=0.6)
        if matches:
            rets.append(matches[0])
        else:
            rets.append(ans[i])
    return rets



real_answers_bert_dp_lower = get_answers(answers_bert_dp_lower)
real_answers_rubert_conv_dp_lower = get_answers(answers_rubert_conv_dp_lower)
real_answers_rubert_conv_dp_notlower =  get_answers(answers_rubert_conv_dp_notlower)
real_answers_rubert_lower = get_answers(answers_rubert_lower)
real_answers_rubert_sen_lower = get_answers(answers_rubert_sen_lower)
real_answers_bert_fin = get_answers(answers_bert_fin)
real_answers_bert_multi = get_answers(answers_bert_multi)
real_answers_bert_fin = get_answers(answers_bert_fin)
real_answers_rubert_sen_dp_lower = get_answers(answers_rubert_sen_dp_lower)


In [ ]:
len(real_answers_bert_xquad_notlower)

7257

In [ ]:
import pandas as pd

answers__06 = pd.DataFrame({
                          'bert_dp_lower':real_answers_bert_dp_lower, 'rubert_conv_dp_lower':real_answers_rubert_conv_dp_lower,
                           'rubert_conv_dp_notlower':real_answers_rubert_conv_dp_notlower,'rubert_lower':real_answers_rubert_lower,
                           'rubert_sen_lower':real_answers_rubert_sen_lower,'bert_fin':real_answers_bert_fin,
                          'bert_multi':real_answers_bert_multi,'bert_fin':real_answers_bert_fin, 'rubert_sen_dp_lower':real_answers_rubert_sen_dp_lower})

answers__06.to_csv('answers_cutoff06.csv')

In [ ]:
import pandas as pd

answers__05 = pd.DataFrame({
                          'bert_dp_lower':real_answers_bert_dp_lower, 'rubert_conv_dp_lower':real_answers_rubert_conv_dp_lower,
                           'rubert_conv_dp_notlower':real_answers_rubert_conv_dp_notlower,'rubert_lower':real_answers_rubert_lower,
                           'rubert_sen_lower':real_answers_rubert_sen_lower,'bert_fin':real_answers_bert_fin,
                          'bert_multi':real_answers_bert_multi,'bert_fin':real_answers_bert_fin, 'rubert_sen_dp_lower':real_answers_rubert_sen_dp_lower})

In [ ]:
real_answers_bert_dp_lower = get_answers(answers_bert_dp_lower)
real_answers_rubert_conv_dp_lower = get_answers(answers_rubert_conv_dp_lower)
real_answers_rubert_conv_dp_notlower =  get_answers(answers_rubert_conv_dp_notlower)
real_answers_rubert_lower = get_answers(answers_rubert_lower)
real_answers_rubert_sen_lower = get_answers(answers_rubert_sen_lower)
real_answers_bert_fin = get_answers(answers_bert_fin)
real_answers_bert_multi = get_answers(answers_bert_multi)
real_answers_bert_fin = get_answers(answers_bert_fin)
real_answers_rubert_sen_dp_lower = get_answers(answers_rubert_sen_dp_lower)


In [ ]:
answers__04.to_csv('answers_cutoff04.csv')

In [ ]:
import pandas as pd

answers__04 = pd.DataFrame({
                          'bert_dp_lower':real_answers_bert_dp_lower, 'rubert_conv_dp_lower':real_answers_rubert_conv_dp_lower,
                           'rubert_conv_dp_notlower':real_answers_rubert_conv_dp_notlower,'rubert_lower':real_answers_rubert_lower,
                           'rubert_sen_lower':real_answers_rubert_sen_lower,'bert_fin':real_answers_bert_fin,
                          'bert_multi':real_answers_bert_multi,'bert_fin':real_answers_bert_fin, 'rubert_sen_dp_lower':real_answers_rubert_sen_dp_lower})

In [ ]:
answers__05

,bert_dp_lower,rubert_conv_dp_lower,rubert_conv_dp_notlower,rubert_lower,rubert_sen_lower,bert_fin,bert_multi,rubert_sen_dp_lower
0,"[косово, 0.9944238693830514]","[косово, 0.9999718868757788]","[косово, 0.9965938602112344]","[косово, 0.057627473616651645]","[косово, 0.999995775038059]","[косово, 0.9999457889143738]","[косово, 0.0758041218550305]","[косово, 0.9999008528005779]"
1,"[sunday mirror, 0.789356265118579]","[sunday mirror, 0.945510378299067]","[sunday mirror, 0.9869288114301938]","[sunday mirror, 0.9932852205806777]","[sunday mirror, 0.9999917874933406]","[sunday mirror, 0.9952558064648346]","[sunday mirror, 0.9991581242474153]","[sunday mirror, 0.948093765525396]"
2,"[греции, 0.9988897478439686]","[греции, 0.9999816182748688]","[греции, 0.8980526563540486]","[греции, 0.24892274801123498]","[греции, 0.999901077362897]","[греции, 0.9997635905240112]","[греции, 0.3326610568477036]","[греции, 0.999061216236093]"
3,"[амир, 0.9251459250336475]","[амир, 0.9149444959009427]","[амир, 0.5637315149250158]","[амир, 0.48603500328961197]","[афганистана., 0.9230469974741327]","[амир, 0.9950145902397293]","[амир, 0.5103474172125624]","[афганистана., 0.8419654892852706]"
4,"[германия, 0.9997232536134742]","[германия, 0.9999981205645674]","[германия, 0.9999873493843309]","[германия, 0.999802781887209]","[германия, 0.9999967309275971]","[германия, 0.9998036527698703]","[германия, 0.9998225043764025]","[германия, 0.9999068728635685]"
...,...,...,...,...,...,...,...,...
7252,"[джигана, 0.9999187932971122]","[джигана, 0.9999546430746388]","[оксана самойлова, 0.7330542772481708]","[джигана, 0.21121041661675485]","[джигана, 0.999992311107574]","[джигана, 0.999970673476162]","[джигана, 0.21734528441454537]","[джигана, 0.9999675790729993]"
7253,"[андрей ермак, 0.5651218721886843]","[сбу, 0.9780637632017597]","[зеленского, 0.9980990734913527]","[ермак, 0.4368291626200772]","[ермак, 0.6424191973275251]","[андрей ермак, 0.9713538396079444]","[зеленского, 0.8764895810411018]","[зеленского, 0.2584079393614943]"
7254,"[чухаджян, 0.9831100846416737]","[чухаджян, 0.9998032596800128]","[чухаджян, 0.9994727762542662]","[чухаджян, 0.2648023419038367]","[чухаджян, 0.9995137112822595]","[чухаджян, 0.9994651076248184]","[чухаджян, 0.3267956695226427]","[чухаджян, 0.997631239733393]"
7255,"[кобешавидзе, 0.7891792446500037]","[кобешавидзе, 0.999974032856808]","[кобешавидзе, 0.9869100407720821]","[кобешавидзе, 0.2230348662347099]","[кобешавидзе, 0.9994882888027529]","[кобешавидзе, 0.9232467696418172]","[кобешавидзе также рассказал, что за 500 долла...","[кобешавидзе, 0.9804096691800159]"


In [ ]:
answers__05.to_csv('answers_cutoff05.csv')

In [ ]:
# еще один ансамбль!
real_an_05 = []
for i in range(len(answers__05[:])):
    st = answers__05.iloc[i]
    s = Counter([x[0].lower() for x in st])
    for x in s:
        if s[x] >= 5:
            real_an_05.append(x)
            break
    else:
        real_an_05.append(sorted(answers__05.iloc[i].values, key=lambda x: -x[1])[0][0].lower())

In [ ]:
# и еще один ансамбль
real_an_06 = []
for i in range(len(answers__06[:])):
    st = answers__06.iloc[i]
    # print(st)
    s = Counter([x[0].lower() for x in st])
    for x in s:
        if s[x] >= 5:
            real_an_06.append(x)
            break
    else:
        real_an_06.append(sorted(answers__06.iloc[i].values, key=lambda x: -x[1])[0][0].lower())

In [ ]:
# в общем ансамблей много не бывает
real_an_04 = []
for i in range(len(answers__04[:])):
    st = answers__04.iloc[i]
    # print(st)
    s = Counter([x[0].lower() for x in st])
    for x in s:
        if s[x] >= 5:
            real_an_04.append(x)
            break
    else:
        real_an_04.append(sorted(answers__04.iloc[i].values, key=lambda x: -x[1])[0][0].lower())

In [ ]:
b = []
# res (4).jsonl - это наш сабмит на 0.8327 (вроде)
'''
for i,k in enumerate(entities_m[:]):
    mathes_be = difflib.get_close_matches(best_answers[i], k, cutoff=0.4)
    if mathes_be:
        best_answers[i] = mathes_be[0]
это ансамбль для сабмита на 0.8327
'''

with open('res (4).jsonl', 'r') as f:
    la = f.readlines()
    # print(la)
    if 'text' in la:
        b.append(la)
    # best_answers = list(f)
# for i in range(len(best_ers)):
#     best_answers.append(best_ers[i])

In [ ]:
b # я сам не знаю что это, но кажется на этой строчке держаться все наши ансамбли

[]

In [ ]:
f =  open('best.jsonl', 'r') # это по идее тот же сабмит на 0.8327

In [ ]:
lls = f.readlines()
# f.readlines()

In [ ]:
len(lls)

29029

In [ ]:
best_answ = []
for i in lls[:]:
    if 'text' in i:
        best_answ.append(i)

In [ ]:
best_answ_ = []
for i in best_answ[:]:
    # best_answ_.append(i.replace(" ",'').replace('"text":', ''))
    best_answ_.append(i.replace('"text":', '').replace('"', ''))

In [ ]:
best_answ_

In [ ]:
best_answers = []
for i in best_answ_[:]:
    best_answers.append(i.replace('\n', '')[5:])
    # print(i.replace('\n', '')[5:])

In [ ]:
best_answers

In [ ]:
'text' in '    "text": "косово"'


True

In [ ]:
f.readline()

'    "text": "косово"\n'

In [ ]:
# корректировка файла в нужный формат

a = open('res (4).jsonl', 'r').read()
# f1 = open('ress.jsonl', 'w')
rs = ''
for i in range(len(a)):
    rs += a[i]
    if i > 1:
        if a[i] == ',' and a[i-1] == '}':
            rs = rs[:-1]
rs = rs.split('\n')
rs = [x[4:] for x in rs]
# rs = .join(rs)

In [ ]:
# это тот самый ансамбль на 0.8327

for i,k in enumerate(entities_m[:]):
    mathes_be = difflib.get_close_matches(best_answers[i], k, cutoff=0.4)
    if mathes_be:
        best_answers[i] = mathes_be[0]

In [ ]:
# еще один ансабмль
for i,k in enumerate(entities_m[:]):
    if real_an_05[i] != best_answers[i]:
        # if len(real_an[i]) > len(best_answers[i]):
        matches_an = difflib.get_close_matches(real_an_05[i], k, cutoff=0.5)
        mathes_be = difflib.get_close_matches(best_answers[i], k, cutoff=0.5)
        if matches_an == mathes_be:
            real_an_05[i] = mathes_be
        else:
            real_an_05[i] = matches_an

        #     real_an[i] = best_answers[i]
        # elif len(real_an[i]) < len(best_answers[i]):
            
        #     matches_an = difflib.get_close_matches(real_an[i], k, cutoff=0.4)
        #     mathes_be = difflib.get_close_matches(best_answers[i], k, cutoff=0.4)
        #     real_an[i] = best_answers[i]
        # print(real_an_04[i], '-'*5, best_answers[i])

In [ ]:
# +1 ансамбль
real_an_03 = []
for i in range(len(answers__)):
    st = answers__.iloc[i]
    s = Counter([x[0].lower() for x in st])
    for x in s:
        if s[x] >= 2:
            real_an_03.append(x)
            break
    else:
        real_an_03.append(sorted(answers__.iloc[i], key=lambda x: -x[1])[0][0].lower())

In [ ]:
real_an_04 == real_an_03

In [ ]:
answq = []
# корректировка файла в нужный формат
for i,l in enumerate(real_an_05):
    a = {'idx': i, 'text':str(l)}
    answq.append(a)
import json
with open('data.jsonl', 'w', encoding='utf-8') as f:
    json.dump(answq, f, ensure_ascii=False, indent=4)

a = open('data.jsonl', 'r').read()
f1 = open('ensemble_with_best_normali_05.jsonl', 'w')
rs = ''
for i in range(len(a)):
    rs += a[i]
    if i > 1:
        if a[i] == ',' and a[i-1] == '}':
            rs = rs[:-1]
rs = rs.split('\n')
rs = [x[4:] for x in rs]
rs = '\n'.join(rs)
f1.write(rs)
f1.close()


In [ ]:
real_an_04

In [ ]:
for i,k in enumerate(best_answers):
    if len(k.split(' ')) > 3:
        print(i, k)

1382 the arctic bath hotel
1641 амурской областной детской клинической больницы
1647 human love doll company
3222 международная федерация футбольных ассоциаций
4571 джорджа оруэлла айлин о'шонесси
5849 россии владимир путин провел телефонный разговор с турецким коллегой реджепом тайипом эрдоганом. об этом сообщается на сайте кремля. в беседе с эрдоганом путин отказал в разделе сирии
5929 новых санжарах полтавской области
6077 палаты общин. историки знали о существовании коридора, так как его уже находили в 1950-х годах. тогда здание восстанавливали после бомбардировок в ходе второй мировой войны, а проход был замурован и забыт. в этот раз на него указала табличка в вестминстерском зале
6591 анастасии волочковой наталья ракицкая


In [ ]:
best_answers[6591] = 'ракицкая'

In [ ]:
test[6591]

{'context': 'pr‐директор анастасии волочковой наталья ракицкая извинилась перед оскорбленной балериной журналисткой и попыталась оправдать подопечную, сославшись на особенности ее характера. об этом сообщается на сайте радиостанции «говорит москва». ракицкая объяснила что волочкова ответила грубо, так как «только проснулась» и не разобралась в ситуации. «она только проснулась, и сразу ваша эсэмэска, вот и реакция такая, понимаете. дело в том, что анастасия не смотрела передачу про эту жену блогера, ее это мало интересует, она даже не знает этой проблемы, понимаете, а вы ее спрашиваете, она не поняла, в чем дело», — отметила pr‐директор. тем не менее, ракицкая осудила балерину за оскорбления внешности корреспондентки и «сделала ей замечание».\n@highlight\npr‐директор волочковой извинилась за балерину перед корреспонденткой «говорит москва»\n@highlight\nволочкову раскритиковали за праздник с сухим льдом\n@highlight\nволочкова отметила праздник с сухим льдом',
 'qas': [{'answers': [{'answ

In [ ]:
entities_m[6591]

['pr',
 'анастасии волочковой наталья ракицкая',
 'волочковой',
 'ракицкая',
 'говорит москва',
 'ракицкая',
 'волочкова',
 'анастасия',
 'pr',
 'ракицкая',
 'pr',
 'волочковой',
 'говорит москва',
 'волочкову',
 'волочкова']

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
! pip install navec

--2021-03-03 22:01:29--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.7MB/s    in 3.0s    

2021-03-03 22:01:33 (16.7 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
from navec import Navec

path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
answers__

,Unnamed: 0,bert_xquad_notlower,distilbert_notlower,bert_dp_lower,rubert_conv_dp_lower,rubert_conv_dp_notlower,rubert_sen_dp_lower
0,0,германии,германии,германии,германии,германии,германии
1,1,хамас,хамас,рф,рф,хамас,рф
2,2,ташкента,ташкента,ташкента,ташкента,ташкента,ташкента
3,3,heckler & koch,heckler & koch,heckler & koch,heckler & koch,сми,heckler & koch
4,4,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер,франк-вальтер штайнмайер
...,...,...,...,...,...,...,...
7572,7572,ирана,ближнем востоке,ирана,иран,ирана,ирана
7573,7573,николаенко,николаенко,николаенко,николаенко,николаенко,николаенко
7574,7574,халифа хафтар,халифа хафтар,халифа хафтар,халифа хафтар,хафтар,халифа хафтар
7575,7575,халифы хафтара,халифы хафтара,халифы хафтара,хафтара,россии,хафтара
